# SediNet: compute sieve size

Demonstration of SediNet using [Panel](https://panel.pyviz.org/index.html)

In [ ]:
from file_select import FileBrowser
from PIL import Image
import random

In [ ]:
import sys
#sys.path.append('../')
from sedinet_models import *
import panel as pn
pn.extension()
#import ipywidgets as widgets

In [ ]:
configfile = 'config_sievedsand_sieve.json'

In [ ]:
import json
# load the user configs
with open('config'+os.sep+configfile) as f:    
  config = json.load(f) 

###===================================================
## user defined variables: proportion of data to use for training (a.k.a. the "train/test split")
base    = int(config["base"]) #minimum number of convolutions in a sedinet convolutional block
csvfile = config["csvfile"] #csvfile containing image names and class values
res_folder = config["res_folder"] #folder containing csv file and that will contain model outputs
name = config["name"] #name prefix for output files
dropout = float(config["dropout"]) 
add_bn = bool(config["add_bn"]) 

vars = [k for k in config.keys() if not np.any([k.startswith('base'), k.startswith('res_folder'), k.startswith('csvfile'), k.startswith('name'), k.startswith('dropout'), k.startswith('add_bn')])]

vars = sorted(vars)

###==================================================

csvfile = os.path.abspath(os.getcwd()+os.sep+res_folder+os.sep+csvfile)

In [ ]:
#csvfile

In [ ]:
## read the data set in, clean and modify the pathnames so they are absolute
df = pd.read_csv(csvfile)
df['files'] = [k.strip() for k in df['files']]
df['files'] = [os.getcwd()+os.sep+f.replace('\\',os.sep) for f in df['files']]    

In [ ]:
models = []
for base in [base-2,base,base+2]:
  weights_path = name+"_base"+str(base)+"_model_checkpoint.hdf5"
  ##==============================================
  ## create a SediNet model to estimate sediment category
  model = make_cont_sedinet(base, vars, add_bn, dropout)
  model.load_weights(os.getcwd()+os.sep+'res'+os.sep+res_folder+os.sep+weights_path)
  models.append(model)

In [ ]:
def get_image():
    #get a random image
    n = random.choice((67364561,67447491,67449081,59858041,94974311,65614911))
    img = 'https://www.allaboutbirds.org/guide/assets/photo/{n}-1280px.jpg'.format(n=n)
    return img

def get_grainsize():
    
    im = Image.open(file_input.value).convert('LA')
    im = im.resize((IM_HEIGHT, IM_HEIGHT))
    im = np.array(im) / 255.0    
    im2 = np.rot90(im)         
    im = np.expand_dims(np.hstack((im[:,:,0], im2[:,:,0])),axis=2) 
    im = np.expand_dims(im, axis=0)

    for v in vars:
      exec(v+'_PT = []')
     
    for model in models:   
      tmp = model.predict(im, batch_size=1)
      exec(vars[0]+'_PT.append(np.asarray(np.squeeze(tmp)))') #.argmax(axis=-1))')

    exec(vars[0]+'_predT = np.squeeze(np.mean(np.asarray('+vars[0]+'_PT), axis=0))')

    Text.object = 'Grain size: '+str(eval(vars[0]+'_predT'))+' microns'
    return Text, Text2

def get_plot():
    im = Image.open(file_input.value)#.convert('LA')
    im = np.array(im) / 255.0    
    plot = plt.imshow(im)
    return plot

def show_message():
    Text2.object = 'Calculated grain size for '+file_input.value +'.....'
    return Text2

def update(event):
    Text2 = show_message()
    jpg_panel.object = file_input.value 
    Text = get_grainsize()
    
##=====    
title = pn.pane.HTML('<h2> SediNet: compute sieve size</h2>')

imfile = 'MardaScience_logo_circle.JPG'
#setup the logo pane
logo = pn.pane.JPG(get_image(), width=100)
logo.object = imfile

update_btn = pn.widgets.Button(name='Calculate Grain Size!',width=100,button_type='primary')
s = pn.pane.HTML('<h5>Compute equivalent sieve diameters directly from an image, without the need for area-to-mass conversion formulas and without even knowing the scale of one pixel. This demonstration has been implemented using a a small population of beach sands</h5>')
file_input = pn.widgets.FileInput()
file_input = pn.widgets.Select(name='Select an image file:', options=list(df.files.values))

Text = pn.panel('Grain size:', width=200, height=60)

Text2 = pn.panel('', width=200, height=60)
    
#setup the jpg pane
jpg_panel = pn.pane.JPG(get_image(), width=300)
jpg_panel.object = file_input.value 

footer = pn.pane.HTML('<h6>Demonstration of SediNet using Panel. Written by Daniel Buscombe, MARDA Science</h6>')

#watch associates a button click with the update function
update_btn.param.watch(update, 'clicks')
#add the widgets to panes
top = pn.Row(file_input) 
mid = pn.Row(update_btn)

app = pn.Column(title, logo, s, top,mid,jpg_panel, Text2, Text,footer)
app.show()